In [1]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [ ]:
#import sys
#!{sys.executable} -m pip install numpy-financial

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import numpy_financial as npf
import datetime as dt
import os
import warnings

In [3]:
warnings.filterwarnings('ignore')
adress = os.getcwd()
adress = adress + '\\'
adress_to_save = adress.replace('input', 'output/data/')
today_date = dt.datetime.today().date()
df = pd.read_csv('Baseline Forecast.csv')
def clean_market(df):
    df = df.drop(['Property Class Name','Property Type', 'Geography Type', 'As Of', 'Forecast Scenario'], axis=1)
    df.columns = [i.lower().replace(' ', '_').replace('/', '') for i in df.columns]
    df = df.melt(id_vars=df.columns[0:5],
           value_vars=df.columns[5:], var_name='Concept', value_name='Value')
    df.columns = [i.lower().replace(' ', '_').replace('/', '') for i in df.columns]
    df = df.dropna()
    df['year'] = df['period'].str.split(' Q').str[0]
    df = df.loc[df['period'].str.contains(' EST') != True]
    df['quarter'] = df['period'].str.split('Q').str[1]#.str.replace(' EST', '')
    df = df.drop(['period'], axis=1)
    df['year'] = df['year'].astype(int)
    df['quarter'] = df['quarter'].astype(int) *3
    df['year'] = df['year'].astype(str)
    df['quarter'] = df['quarter'].astype(str)      
    df['date'] = pd.to_datetime(   (df['year'] + '-' + df['quarter'] + '-28'))
    df['date'] = df['date'] + pd.offsets.MonthEnd(0)
    df.drop(['year', 'quarter'], axis=1, inplace=True)
    df['state'] = df['geography_name'].str.replace(' USA', '').str[-2:].str.strip()
    df['city'] = df['geography_name'].str.replace(' USA', '').str[:-4].str.strip()
    df['city'] = np.where(df['city'].str.contains('-'), df['city'].str.split('-').str[0], df['city'])
    df['state'] = df['state'].map(abbrev_to_us_state)
    df['market'] = df['city'] + ',' + df['state']
    df = df.drop(['geography_name'], axis=1)
    return df
df = clean_market(df)
df

,slice,geography_code,cbsa_code,concept,value,date,state,city,market
0,Class A,G2-10180,10180,absorption_%,-0.002083,2010-03-31,Texas,Abilene,"Abilene,Texas"
1,Class A,G2-10180,10180,absorption_%,0.000000,2010-06-30,Texas,Abilene,"Abilene,Texas"
2,Class A,G2-10180,10180,absorption_%,0.004166,2010-09-30,Texas,Abilene,"Abilene,Texas"
3,Class A,G2-10180,10180,absorption_%,-0.002083,2010-12-31,Texas,Abilene,"Abilene,Texas"
4,Class A,G2-10180,10180,absorption_%,-0.004166,2011-03-31,Texas,Abilene,"Abilene,Texas"
...,...,...,...,...,...,...,...,...,...
5680075,Class C,G2-49740,49740,vacancy_rate,0.059614,2029-03-31,Arizona,Yuma,"Yuma,Arizona"
5680076,Class C,G2-49740,49740,vacancy_rate,0.059458,2029-06-30,Arizona,Yuma,"Yuma,Arizona"
5680077,Class C,G2-49740,49740,vacancy_rate,0.059232,2029-09-30,Arizona,Yuma,"Yuma,Arizona"
5680078,Class C,G2-49740,49740,vacancy_rate,0.059040,2029-12-31,Arizona,Yuma,"Yuma,Arizona"


In [ ]:
#import sys
#print(sys.executable)
#!{sys.executable} -m pip install geopy

c:\Users\DaniloRodriguezArang\anaconda3\python.exe


In [4]:
populations = df.query('date == "2024-12-31" & concept == "population"').copy()
populations = populations[['state', 'city', 'market', 'value']].query('value > 500000').drop_duplicates()

markets = df.copy().loc[df['market'].isin(populations['market'])]

from geopy.geocoders import Nominatim


import requests
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="JuanVillalba_Casa")
import time

# # Función para geocodificar ciudad y estado (nombres completos)
def geocode_city_state(city, state):
    time.sleep(1)  # Pausa de 1 segundo entre solicitudes para evitar el bloqueo
    location = geolocator.geocode(f"{city}, {state}")
    if location:
        return location.latitude, location.longitude
    else:
        return None, None
    
populations['latitude'], populations['longitude'] = zip(*populations.apply(lambda x: geocode_city_state(x['city'], x['state']), axis=1))

#para los ultimos 5 años

In [7]:
##### Versión 2.0 


ltv = 0.7
opex = 0.54
int_rate = 0.0217
spread = 0.03
net = int_rate + spread
term = 10

date_calculated = dt.datetime(2015, 6, 30)
yrs_after = date_calculated + pd.DateOffset(years=term)

markets_list = markets['market'].unique()
slices = ['Class A', 'Class B', 'Class C']

# Para guardar resultados
cashflows = []
tables = []
equilibriums = []
summaries = []

for market in markets_list:
    for slice_ in slices:
        try:
            # Filtrar datos para mercado, slice y rango de fecha
            data = markets.query('market == @market & slice == @slice_ & date >= @date_calculated & date <= @yrs_after')
            
            if data.empty:
                print(f'No hay datos para {market} {slice_}')
                continue
            
            # Validar que contenga fechas completas para 10 años trimestrales (aprox 41 trimestres: 10*4 + 1)
            expected_len = term * 4 + 1
            unique_dates = data['date'].sort_values().unique()
            if len(unique_dates) < expected_len:
                print(f'Fechas insuficientes para {market} {slice_} ({len(unique_dates)} registros, esperado {expected_len})')
                continue
            
            # Extraer datos necesarios
            cbsa = data['cbsa_code'].iloc[0]
            geo_code = data['geography_code'].iloc[0]
            state = data['state'].iloc[0]
            city = data['city'].iloc[0]
            population = data.loc[data['concept'] == 'population', 'value'].iloc[0]
            
            # Precio actual (fecha inicial)
            current_price = data.loc[
                (data['concept'] == 'market_sale_price_per_unit') & 
                (data['date'] == date_calculated), 'value']
            
            if current_price.empty:
                print(f'No hay precio actual para {market} {slice_}')
                continue
            
            current_price = current_price.iloc[0]
            
            loan = current_price * ltv
            equity = current_price - loan
            
            # Pivot tabla para conceptos vs fecha
            transformed = data.pivot_table(index='date', columns='concept', values='value').sort_index()
            
            # Validar fechas límite en el pivot
            if transformed.index.min() != date_calculated or transformed.index.max() < yrs_after:
                print(f'Fechas del pivot no cumplen rango para {market} {slice_}')
                continue
            
            # Filtrar para tener 1 dato cada año (usar trimestrales: tomar el trimestre final de cada año)
            # Aquí puedes ajustar según tus datos trimestrales o anuales
            transformed_annual = transformed.resample('A', axis=0).last()
            if len(transformed_annual) != term + 1:
                print(f'No hay suficientes datos anuales para {market} {slice_}')
                continue
            
            equilibrium = transformed_annual[['absorption_units_12_mo', 'demand_units', 'inventory_units', 'net_delivered_units_12_mo']].copy()
            equilibrium['horizon'] = f'{term} Yrs ({date_calculated.year}-{yrs_after.year})'
            demand_vs_supply = equilibrium['demand_units'].sum() / equilibrium['inventory_units'].sum()
            demand_yoy_growth = (equilibrium['demand_units'].iloc[-1] / equilibrium['demand_units'].iloc[0])**(1/term) - 1
            supply_yoy_growth = (equilibrium['inventory_units'].iloc[-1] / equilibrium['inventory_units'].iloc[0])**(1/term) - 1
            
            table_save = transformed_annual[['market_cap_rate', 'market_effective_rent_growth_12_mo',
                                             'market_effective_rentunit','market_sale_price_growth', 
                                             'market_sale_price_per_unit','occupancy_rate']].copy()
            table_save['revenue'] = table_save['market_effective_rentunit'] * 12 * table_save['occupancy_rate']
            table_save.loc[date_calculated, 'revenue'] = 0
            table_save['opex'] = opex
            table_save.loc[date_calculated, 'opex'] = np.nan
            table_save['noi'] = table_save['revenue'] * (1 - table_save['opex'])
            table_save['noi'] = table_save['noi'].fillna(0)
            table_save['market'] = market
            table_save['horizon'] = f'{term} Yrs ({date_calculated.year}-{yrs_after.year})'
            
            cashflow = pd.DataFrame(index=table_save.index)
            cashflow.loc[date_calculated, 'equity'] = -equity
            cashflow['price'] = transformed_annual['market_sale_price_per_unit']
            cashflow['revenue'] = table_save['revenue']
            cashflow['noi'] = table_save['noi']
            cashflow['debt_payment'] = -loan * net
            cashflow.loc[date_calculated, 'debt_payment'] = 0
            cashflow.loc[yrs_after, 'loan_payoff'] = -loan
            cashflow.loc[yrs_after, 'valuation'] = cashflow.loc[yrs_after, 'price']
            cashflow = cashflow.fillna(0)
            cashflow['cashflow'] = cashflow['equity'] + cashflow['noi'] + cashflow['debt_payment'] + cashflow['loan_payoff'] + cashflow['valuation']
            cashflow['horizon'] = f'{term} Yrs ({date_calculated.year}-{yrs_after.year})'
            
            irr = npf.irr(cashflow['cashflow'])
            npv = npf.npv(net, cashflow['cashflow'])
            
            summary = pd.DataFrame({
                'date_calculated': [date_calculated],
                'date_end': [yrs_after],
                'slice': [slice_],
                'state': [state],
                'city': [city],
                'market': [market],
                'cbsa': [cbsa],
                'geo_code': [geo_code],
                'population': [population],
                'current_price': [current_price],
                'loan': [loan],
                'equity': [equity],
                'market_cagr': [(transformed_annual['market_sale_price_per_unit'].iloc[-1] / transformed_annual['market_sale_price_per_unit'].iloc[0])**(1/term) - 1],
                'noi_cap_rate_compounded': [(cashflow['noi'].sum()/term) / current_price],
                'fixed_interest_rate': [(cashflow['debt_payment'].sum() / term) / -loan],
                'operation_cashflow': [(cashflow['noi'].sum()/term) / current_price - (cashflow['debt_payment'].sum() / term) / -loan],
                'market_cap_appreciation_bp': [(table_save['market_cap_rate'].iloc[0] - table_save['market_cap_rate'].iloc[-1]) * 10000],
                'irr': [irr],
                'npv': [npv],
                'equity_multiple': [(cashflow['cashflow'].sum() + equity) / equity],
                'demand_vs_supply': [demand_vs_supply],
                'demand_yoy_growth': [demand_yoy_growth],
                'supply_yoy_growth': [supply_yoy_growth],
                'horizon': [f'{term} Yrs ({date_calculated.year}-{yrs_after.year})']
            })
            
            # Guardar resultados en listas
            cashflows.append(cashflow.assign(market=market, date_cashflow=date_calculated, slice=slice_))
            tables.append(table_save.assign(market=market, date_calculated=date_calculated, slice=slice_))
            equilibriums.append(equilibrium.assign(market=market, date_calculated=date_calculated, slice=slice_))
            summaries.append(summary)
            
            print(f'Datos guardados para {market} {slice_}')
        
        except Exception as e:
            print(f'Error en {market} {slice_}: {e}')

# Convertir listas a DataFrames finales si quieres
cashflows_df = pd.concat(cashflows)
tables_df = pd.concat(tables)
equilibriums_df = pd.concat(equilibriums)
summaries_df = pd.concat(summaries)


Datos guardados para Akron,Ohio Class A
Datos guardados para Akron,Ohio Class B
Datos guardados para Akron,Ohio Class C
Datos guardados para Albany,New York Class A
Datos guardados para Albany,New York Class B
Datos guardados para Albany,New York Class C
Datos guardados para Albuquerque,New Mexico Class A
Datos guardados para Albuquerque,New Mexico Class B
Datos guardados para Albuquerque,New Mexico Class C
Datos guardados para Atlanta,Georgia Class A
Datos guardados para Atlanta,Georgia Class B
Datos guardados para Atlanta,Georgia Class C
Datos guardados para Augusta,Georgia Class A
Datos guardados para Augusta,Georgia Class B
Datos guardados para Augusta,Georgia Class C
Datos guardados para Austin,Texas Class A
Datos guardados para Austin,Texas Class B
Datos guardados para Austin,Texas Class C
Datos guardados para Bakersfield,California Class A
Datos guardados para Bakersfield,California Class B
Datos guardados para Bakersfield,California Class C
Datos guardados para Baltimore,Maryla

In [9]:
### Version 2.0 segunda parte
markets_list = markets['market'].unique()
geos = markets_list

ltv = 0.7
opex = 0.54
int_rate = 0.0415
spread = 0.03
net = int_rate + spread
term = 5

general_parameters = pd.DataFrame(index=['ltv', 'opex', 'int_rate', 'spread', 'net_rate', 'term'],
                                    data={'value': [ltv, opex, int_rate, spread, net, term]})

date_calculated = dt.datetime(2025, 3, 31)

cashflows = []
tables = []
equilibriums = []
summaries = []

for i in geos: 
    for slice_ in ['Class A', 'Class B', 'Class C']:     
        try:
            yrs_after = date_calculated + pd.offsets.DateOffset(years=term)
            data = markets.query('market == @i & date >= @date_calculated & date <= @yrs_after & slice == @slice_')

            if data.empty:
                print('No hay datos para ', i, slice_)
                continue
            
            cbsa = data['cbsa_code'].unique()[0]
            geo_code = data['geography_code'].unique()[0]
            population = data.loc[data['concept'] == 'population', 'value'].values[0]
            state = data['state'].unique()[0]
            city = data['city'].unique()[0]
            current_price = data.loc[(data['concept'] == 'market_sale_price_per_unit') & 
                                        (data['date'] == date_calculated), 'value'].values[0]
            loan = current_price * ltv
            equity = current_price - loan

            transformed = data[['date', 'concept', 'value']].pivot_table(index='date', columns='concept', values='value')
            transformed = transformed.iloc[::4, :]  # cada 4 trimestres = 1 año
            
            condition = (transformed.index[0] == date_calculated) and (transformed.index[-1] == yrs_after) and (len(transformed) == term + 1)

            if condition:
                equilibrium = transformed[['absorption_units_12_mo', 'demand_units', 'inventory_units', 'net_delivered_units_12_mo']].copy()
                equilibrium['horizon'] = f'{term} Yrs Ahead ({date_calculated.year}-{yrs_after.year})'
                demand_vs_supply = equilibrium['demand_units'].sum() / equilibrium['inventory_units'].sum()
                demand_yoy_growth = (equilibrium['demand_units'].iloc[-1] / equilibrium['demand_units'].iloc[0])**(1/term) - 1
                supply_yoy_growth = (equilibrium['inventory_units'].iloc[-1] / equilibrium['inventory_units'].iloc[0])**(1/term) - 1

                table_save = transformed[['market_cap_rate', 'market_effective_rent_growth_12_mo',
                                        'market_effective_rentunit','market_sale_price_growth', 
                                        'market_sale_price_per_unit','occupancy_rate']].copy()
                table_save['revenue'] = table_save['market_effective_rentunit'] * 12 * table_save['occupancy_rate']
                table_save.loc[date_calculated, 'revenue'] = 0
                table_save['opex'] = opex
                table_save.loc[date_calculated, 'opex'] = np.nan
                table_save['noi'] = table_save['revenue'] * (1 - table_save['opex'])
                table_save['noi'] = table_save['noi'].fillna(0)
                table_save['market'] = i
                table_save['horizon'] = f'{term} Yrs Ahead ({date_calculated.year}-{yrs_after.year})'

                cashflow = pd.DataFrame(index=table_save.index)
                cashflow.loc[date_calculated, 'equity'] = -equity
                cashflow['price'] = transformed['market_sale_price_per_unit']
                cashflow['revenue'] = table_save['revenue']
                cashflow['noi'] = table_save['noi']
                cashflow['debt_payment'] = -loan * net
                cashflow.loc[date_calculated, 'debt_payment'] = 0
                cashflow.loc[yrs_after, 'loan_payoff'] = -loan
                cashflow.loc[yrs_after, 'valuation'] = cashflow.loc[yrs_after, 'price']

                cashflow = cashflow.fillna(0)
                cashflow['cashflow'] = cashflow['equity'] + cashflow['noi'] + cashflow['debt_payment'] + cashflow['loan_payoff'] + cashflow['valuation']
                cashflow['horizon'] = f'{term} Yrs Ahead ({date_calculated.year}-{yrs_after.year})'

                irr = npf.irr(cashflow['cashflow'])
                npv = npf.npv(net, cashflow['cashflow'])

                summary = pd.DataFrame({
                    'date_calculated': [date_calculated],
                    'date_end': [yrs_after],
                    'slice': [slice_],
                    'state': [state],
                    'city': [city],
                    'market': [i],
                    'cbsa': [cbsa],
                    'geo_code': [geo_code],
                    'population': [population],
                    'current_price': [current_price],
                    'loan': [loan],
                    'equity': [equity],
                    'market_cagr': [(transformed['market_sale_price_per_unit'].iloc[-1] / transformed['market_sale_price_per_unit'].iloc[0])**(1/term) - 1],
                    'noi_cap_rate_compounded': [(cashflow['noi'].sum()/term) / current_price],
                    'fixed_interest_rate': [(cashflow['debt_payment'].sum() / term) / -loan],
                    'operation_cashflow': [(cashflow['noi'].sum()/term) / current_price - (cashflow['debt_payment'].sum() / term) / -loan],
                    'market_cap_appreciation_bp': [(table_save['market_cap_rate'].iloc[0] - table_save['market_cap_rate'].iloc[-1]) * 10000],
                    'irr': irr,
                    'npv': npv,
                    'equity_multiple': [(cashflow['cashflow'].sum() + equity) / equity],
                    'demand_vs_supply': demand_vs_supply,
                    'demand_yoy_growth': demand_yoy_growth,
                    'supply_yoy_growth': supply_yoy_growth,
                    'horizon': f'{term} Yrs Ahead ({date_calculated.year}-{yrs_after.year})'
                })

                cashflows.append(cashflow.assign(market=i, date_cashflow=date_calculated, slice=slice_))
                tables.append(table_save.assign(market=i , date_calculated=date_calculated, slice=slice_))
                equilibriums.append(equilibrium.assign(market=i, date_calculated=date_calculated, slice=slice_))
                summaries.append(summary)

                print('Datos guardados para ', i, slice_)

            else:
                print('No cumplieron los requisitos ', i, slice_)

        except Exception as e:
            print(f'Error para {i} {slice_}: {e}')

# Concatenar resultados si quieres
cashflows_df = pd.concat(cashflows)
tables_df = pd.concat(tables)
equilibriums_df = pd.concat(equilibriums)
summaries_df = pd.concat(summaries)


Datos guardados para  Akron,Ohio Class A
Datos guardados para  Akron,Ohio Class B
Datos guardados para  Akron,Ohio Class C
Datos guardados para  Albany,New York Class A
Datos guardados para  Albany,New York Class B
Datos guardados para  Albany,New York Class C
Datos guardados para  Albuquerque,New Mexico Class A
Datos guardados para  Albuquerque,New Mexico Class B
Datos guardados para  Albuquerque,New Mexico Class C
Datos guardados para  Atlanta,Georgia Class A
Datos guardados para  Atlanta,Georgia Class B
Datos guardados para  Atlanta,Georgia Class C
Datos guardados para  Augusta,Georgia Class A
Datos guardados para  Augusta,Georgia Class B
Datos guardados para  Augusta,Georgia Class C
Datos guardados para  Austin,Texas Class A
Datos guardados para  Austin,Texas Class B
Datos guardados para  Austin,Texas Class C
Datos guardados para  Bakersfield,California Class A
Datos guardados para  Bakersfield,California Class B
Datos guardados para  Bakersfield,California Class C
Datos guardados 

In [11]:
cashflows_df = pd.concat(cashflows)
cashflows_df.query('market == "Chicago,Illinois" & slice == "Class C"')

,equity,price,revenue,noi,debt_payment,loan_payoff,valuation,cashflow,horizon,market,date_cashflow,slice
date,,,,,,,,,,,,
2025-03-31,-43994.79525,146649.3175,0.000000,0.000000,0.000000,0.00000,0.0000,-43994.795250,5 Yrs Ahead (2025-2030),"Chicago,Illinois",2025-03-31,Class C
2026-03-31,0.00000,159346.7021,16084.029661,7398.653644,-7339.798341,0.00000,0.0000,58.855303,5 Yrs Ahead (2025-2030),"Chicago,Illinois",2025-03-31,Class C
2027-03-31,0.00000,169216.9797,16651.409045,7659.648161,-7339.798341,0.00000,0.0000,319.849820,5 Yrs Ahead (2025-2030),"Chicago,Illinois",2025-03-31,Class C
2028-03-31,0.00000,177212.3784,17081.447312,7857.465764,-7339.798341,0.00000,0.0000,517.667423,5 Yrs Ahead (2025-2030),"Chicago,Illinois",2025-03-31,Class C
2029-03-31,0.00000,185208.0789,17444.602334,8024.517074,-7339.798341,0.00000,0.0000,684.718733,5 Yrs Ahead (2025-2030),"Chicago,Illinois",2025-03-31,Class C
2030-03-31,0.00000,193328.1545,17790.639453,8183.694148,-7339.798341,-102654.52225,193328.1545,91517.528057,5 Yrs Ahead (2025-2030),"Chicago,Illinois",2025-03-31,Class C


In [12]:
-7178.113523/-100393.19612

0.07150000000418355

In [13]:
ltv = 0.7
opex = 0.54
int_rate = 0.0156
spread = 0.03
net = int_rate + spread
term = 5

general_parameters = pd.DataFrame(index=['ltv', 'opex', 'int_rate', 'spread', 'net_rate', 'term'],
                                    data={'value': [ltv, opex, int_rate, spread, net, term]})



ltv = 0.7
opex = 0.54
int_rate = 0.0217
spread = 0.03
net = int_rate + spread
term = 10

general_parameters_2 = pd.DataFrame(index=['ltv', 'opex', 'int_rate', 'spread', 'net_rate', 'term'],
                                    data={'value': [ltv, opex, int_rate, spread, net, term]})




ltv = 0.7
opex = 0.54
int_rate = 0.0415
spread = 0.03
net = int_rate + spread
term = 5

general_parameters_3 = pd.DataFrame(index=['ltv', 'opex', 'int_rate', 'spread', 'net_rate', 'term'],
                                    data={'value': [ltv, opex, int_rate, spread, net, term]})

In [15]:
summaries_df = pd.concat(summaries)
summaries_df['latitude'] = summaries_df['market'].map(populations.set_index('market')['latitude'])
summaries_df['longitude'] = summaries_df['market'].map(populations.set_index('market')['longitude'])
summaries_df

,date_calculated,date_end,slice,state,city,market,cbsa,geo_code,population,current_price,...,market_cap_appreciation_bp,irr,npv,equity_multiple,demand_vs_supply,demand_yoy_growth,supply_yoy_growth,horizon,latitude,longitude
0,2025-03-31,2030-03-31,Class A,Ohio,Akron,"Akron,Ohio",10420,G2-10420,7.023969e+05,122492.2969,...,69.453642,0.154928,15704.991452,1.987221,0.787992,0.023666,0.024076,5 Yrs Ahead (2025-2030),41.083064,-81.518485
0,2025-03-31,2030-03-31,Class B,Ohio,Akron,"Akron,Ohio",10420,G2-10420,7.023969e+05,98156.2062,...,69.767535,0.215498,22241.757091,2.408572,0.936014,0.010463,0.011027,5 Yrs Ahead (2025-2030),41.083064,-81.518485
0,2025-03-31,2030-03-31,Class C,Ohio,Akron,"Akron,Ohio",10420,G2-10420,7.023969e+05,62085.0183,...,70.265085,0.228205,15164.772023,2.476661,0.946861,-0.000533,0.000972,5 Yrs Ahead (2025-2030),41.083064,-81.518485
0,2025-03-31,2030-03-31,Class A,New York,Albany,"Albany,New York",10580,G2-10580,9.111790e+05,268188.6780,...,66.400766,0.132797,26979.289818,1.907062,0.957380,0.037751,0.043020,5 Yrs Ahead (2025-2030),42.651167,-73.754968
0,2025-03-31,2030-03-31,Class B,New York,Albany,"Albany,New York",10580,G2-10580,9.111790e+05,189038.4980,...,66.111088,0.153783,25254.844250,2.038654,0.951268,0.014493,0.012637,5 Yrs Ahead (2025-2030),42.651167,-73.754968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2025-03-31,2030-03-31,Class B,Massachusetts,Worcester,"Worcester,Massachusetts",49340,G2-49340,1.002770e+06,269615.8289,...,66.117682,0.158819,39367.561389,2.106689,0.946032,0.029139,0.023962,5 Yrs Ahead (2025-2030),42.262562,-71.801888
0,2025-03-31,2030-03-31,Class C,Massachusetts,Worcester,"Worcester,Massachusetts",49340,G2-49340,1.002770e+06,154367.0141,...,66.498443,0.190943,30324.229541,2.307657,0.971536,0.000653,-0.000183,5 Yrs Ahead (2025-2030),42.262562,-71.801888
0,2025-03-31,2030-03-31,Class A,Ohio,Youngstown,"Youngstown,Ohio",49660,G2-49660,5.330289e+05,167809.0878,...,69.584474,0.187300,30892.714446,2.237378,0.985047,-0.003023,0.000000,5 Yrs Ahead (2025-2030),41.103579,-80.652016
0,2025-03-31,2030-03-31,Class B,Ohio,Youngstown,"Youngstown,Ohio",49660,G2-49660,5.330289e+05,77489.8252,...,69.696978,0.223340,18521.310063,2.460383,0.971800,-0.001728,0.000987,5 Yrs Ahead (2025-2030),41.103579,-80.652016


In [17]:
mask = (
    df['market'].isin(summaries_df['market'].unique()) & 
    df['concept'].isin(["market_sale_price_per_unit", "occupancy_rate", "market_effective_rentunit"])
)
df.loc[mask].to_csv(adress_to_save + 'CoStar_cleaned.csv', index=False)


In [19]:
pd.concat(cashflows).to_csv(adress_to_save + 'cashflows.csv', index=False)
pd.concat(tables).to_csv(adress_to_save + 'tables.csv', index=False)
pd.concat(equilibriums).to_csv(adress_to_save + 'equilibriums.csv', index=False)
pd.concat(summaries).to_csv(adress_to_save + 'summaries.csv', index=False)

# Si populations y general_parameters ya son DataFrames, ok:
populations.to_csv(adress_to_save + 'populations.csv', index=False)
general_parameters.to_csv(adress_to_save + 'general_parameters.csv', index=False)
general_parameters_2.to_csv(adress_to_save + 'general_parameters_2.csv', index=False)
general_parameters_3.to_csv(adress_to_save + 'general_parameters_3.csv', index=False)
